# cuGraph in primerjava z NetworkX

V tej enoti se boste seznanili s knjižnico cuGraph (CUDA Graph), ki je del ogrodja NVIDIA RAPIDS in je namenjena pospeševanju algoritmov nad grafi. Preizkusili boste delovanje nekaterih najpogostejših algoritmov nad grafi, pri čemer boste uporabljali velike podatkovne zbirke (velepodatke). Knjižnica cuGraph izvaja algoritme nad grafi na grafičnih procesnih enotah (GPE) in tako omogoča pospeševanje teh algoritmov. Pohitritev izvajanja algoritmov nad grafi boste preverili s primerjavo izvedbe na centralni procesorski enoti (CPE). To boste storili s pomočjo knjižnice NetworkX. Kot boste videli v nadaljevanju je koda za izvedbo algoritmov nad grafi zelo podobna. Torej, če ste že vajeni uporabljati algoritme nad grafi na CPE s knjižnico NetworkX, so potrebne le minimalne spremembe v kodi in algoritem se lahko s cuGraph izvaja hitreje na GPE.

V nadaljevanju boste podrobneje analizirali uporabo in hitrost izvajanja algoritmov [PageRank](https://en.wikipedia.org/wiki/PageRank) in [HITS](https://en.wikipedia.org/wiki/HITS_algorithm).

**Preden začnemo, izvedite spodnjo celico, ki vam namesti ogrodje NVIDIA RAPIDS v okolje Google Colab.**

In [1]:
!rm -rf scripts
!rm -rf rapids-csp-utils
!mkdir scripts

install_script = """
#/bin/bash

echo "Installing NVIDIA RAPIDS to Google Colab ..."

# Check GPU
echo "Checking GPU ..."
nvidia-smi

# Clone Git repository
echo 'Cloning NVIDIA RAPIDS Git repository (https://github.com/rapidsai/rapidsai-csp-utils.git)...'
rm -rf rapidsai-csp-utils
git clone https://github.com/rapidsai/rapidsai-csp-utils.git

# Install using pip
echo 'Installing NVIDIA RAPIDS using pip ...'
python /content/rapidsai-csp-utils/colab/pip-install.py

# Post-install check
echo "Checking NVIDIA RAPIDS installation ..."
python /content/scripts/post-install-check.py
"""

post_install_script = """
import cudf
import cuml
import cugraph
import cuspatial
import cuxfilter

print(f\'cuDF: {cudf.__version__}\\ncuML: {cuml.__version__}\\ncuGraph: {cugraph.__version__}\\ncuSpatial: {cuspatial.__version__}\\ncuxfilter: {cuxfilter.__version__}\')
"""

!echo "{install_script}" >> ./scripts/install-rapids-colab.sh
!echo "{post_install_script}" >> ./scripts/post-install-check.py

!bash scripts/install-rapids-colab.sh

Installing NVIDIA RAPIDS to Google Colab ...
Checking GPU ...
Fri Jul  5 11:02:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----

Knjižnico NetworkX preprosto namestimo s pomočjo orodja ``pip`` ali pa na alternativne načine, ki so podrobneje opisani v [dokumentaciji](https://networkx.org/documentation/stable/install.html). V tej enoti se bomo osredotočili na algoritme za analizo povezav (ang. link analysis), seveda pa knjižnica NetworkX podpira [širok nabor drugih koristnih algoritmov in operacij nad grafi](https://networkx.org/documentation/stable/reference/index.html).

In [2]:
!pip install networkx

## Podatkovna zbirka

Za primerjavo cuGraph in NetworkX bomo uporabili [podatkovno zbirko ameriških patentov ``cit-Patents``](https://snap.stanford.edu/data/cit-Patents.html), ki so podani v obliki grafa. Podatkovna zbirka je del virov [SNAP](https://snap.stanford.edu/) (Stanford Network Analysis Project) in ima 3,7 mio vozlišč in 16,5 mio povezav. Prenesimo podatkovno zbirko in jo pripravimo za uporabo.

In [3]:
# prenos SNAP cit-Patents
!wget https://snap.stanford.edu/data/cit-Patents.txt.gz

!gzip -dk cit-Patents.txt.gz

--2024-07-05 11:09:09--  https://snap.stanford.edu/data/cit-Patents.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85139832 (81M) [application/x-gzip]
Saving to: ‘cit-Patents.txt.gz’

cit-Patents.txt.gz  100%[===================>]  81.20M  20.3MB/s    in 7.9s    

2024-07-05 11:09:18 (10.2 MB/s) - ‘cit-Patents.txt.gz’ saved [85139832/85139832]



## Analiza povezav brez pospeševanja (NetworkX)

V tem delu enote bomo naložili podatkovno zbirko v obliki grafa s pomočjo knjižnice Pandas, nato pa bomo preizkusili delovanje algoritmov PageRank in HITS, ki ju bomo izvajali s knjižnico NetworkX. Pri tem bomo merili čas izvajanja in preverili rezultat.

### Nalaganje podatkovne zbirke (Pandas)

Najprej naložimo podatkovno zbirko v podatkovno strkuturo Pandas DataFrame, nato pa jo bomo s funkcijo NetworkX ``from_pandas_edgelist`` pretvorili v graf. Pri tem bomo upoštevali izvorna in ciljna vozlišča (``src`` in ``dst``) ter jim nastavili ustrezni podatkovni tip (int32).

In [4]:
import networkx as nx
import pandas as pd
import scipy.sparse.linalg # nekateri algoritmi zahtevajo vključitev te knjižnice

# nalaganje podatkovne zbirke v Pandas DataFrame
%time edgelist_cpu = pd.read_csv('cit-Patents.txt', skiprows=4, delimiter='\t', names=['src', 'dst'], dtype={'src': 'int32', 'dst': 'int32'})

CPU times: user 4.84 s, sys: 439 ms, total: 5.28 s
Wall time: 9.16 s


Nalaganje podatkovne zbirke je trajalo 9,16s. Na tem mestu lahko podatkovno zbirko pretvorimo v podatkovno strukturo [Graph](https://networkx.org/documentation/stable/reference/classes/graph.html).

In [5]:
# pretvorba v NetworkX Graph
%time G_cpu = nx.from_pandas_edgelist(edgelist_cpu, source='src', target='dst', create_using=nx.DiGraph) # nx.DiGraph pomeni, da gre za usmerjen graf (Directed Graph)

CPU times: user 1min 14s, sys: 3.45 s, total: 1min 18s
Wall time: 1min 19s


Pretvorba v graf je trajala 1min 19s. Prepričajmo se, da je graf uspešno pripravljen za nadaljnjo obdelavo.

In [6]:
print('Graf NetworkX')
print(f'Vozlišča: {G_cpu.number_of_nodes()} \nPovezave: {G_cpu.number_of_edges()}')

Graf NetworkX
Vozlišča: 3774768 
Povezave: 16518948


### PageRank (NetworkX)

Zdaj, ko imamo pripravljen graf, lahko nad njim izvajamo analizo povezav. V tem delu enote bomo to storili z algoritmom PageRank. Algoritem PageRank za vsa vozlišča izračuna range posameznih vozlišč (v našem primeru so to patenti), glede na število njihovih povezav (v našem primeru so to omembe v drugih patentih). Ker je naš graf ogromen (3,7 mio vozlišč) lahko pričakujemo daljši čas izvedbe algoritma. Algoritem PageRank zaženimo s funkcijo [``pagerank()``](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pagerank), pri tem pa merimo čas izvedbe.

In [7]:
%time pagerank_cpu = nx.pagerank(G_cpu)

CPU times: user 1min 38s, sys: 3.61 s, total: 1min 42s
Wall time: 1min 42s


Izvedba algoritma PageRank je trajala 1min 42s. Preverimo še 5 najbolje rangiranih vozlišč:

In [8]:
# urejanje po rangu in izpis prvih 5 vozlišč
dict(sorted(pagerank_cpu.items(), key = lambda x: x[1], reverse = True)[:5])

{4683195: 3.6735963425315776e-05,
 4683202: 3.0504951999092532e-05,
 4812599: 1.635319641970828e-05,
 4358535: 1.6013219648142164e-05,
 4237224: 1.5936895928705412e-05}

### HITS (NetworkX)

Poskusimo še z algoritmom HITS, ki je alternativa algoritmu PageRank. Algoritem HITS zaženimo s funkcijo [``hits()``](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.hits_alg.hits.html#networkx.algorithms.link_analysis.hits_alg.hits) in ponovno merimo čas izvedbe.

In [9]:
%time hits_cpu = nx.hits(G_cpu)

CPU times: user 1min 38s, sys: 7.67 s, total: 1min 46s
Wall time: 1min 39s


Izvedba algoritma HITS je trajala 1min 39s. Poglejmo še 5 vozlišč z najvišjo oceno avtoritete:

In [10]:
authority_scores = hits_cpu[1] # algoritem vrne Tuple; prvi element je slovar z vrednostmi
dict(sorted(authority_scores.items(), key = lambda x: x[1], reverse = True)[:5])

{4308400: 0.0018611577337709774,
 4318791: 0.0018610879127413111,
 4058400: 0.0018598637349770148,
 4284485: 0.0018558819415496715,
 3887450: 0.0018547311772334548}

## Analiza povezav s pospeševanjem (cuDF in cuGraph)

Nalaganje podatkovne zbirke, pretvorbo v graf in izvedbo algoritmov PageRank ter HITS smo v prejšnjem sklopu enote izvajali izključno na CPE. V naslednjem sklopu bomo izvedli enako le, da bomo tokrat uporabili pospeševanje na GPE s pomočjo ogrodja NVIDIA RAPIDS.

### Nalaganje podatkovne zbirke (cuDF)

Tokrat bomo podatkovno zbirko s knjižnico cuDF naložili neposredno v pomnilnik GPE. Vključimo cuDF in cuGraph in izmerimo čas nalaganja podatkovne zbirke v podatkovno strukturo cuDF DataFrame.

In [11]:
import cudf
import cugraph as cg

%time edgelist_gpu = cudf.read_csv('cit-Patents.txt', skiprows=4, delimiter='\t', names=['src', 'dst'], dtype={'src': 'int32', 'dst': 'int32'})

CPU times: user 206 ms, sys: 163 ms, total: 369 ms
Wall time: 461 ms


Nalaganje zbirke je tokrat trajalo 461ms kar je približno **20x hitreje** kot prej. Izvedimo še pretvorbo v graf.

In [18]:
G_gpu = cg.Graph(directed=True)
%time G_gpu.from_cudf_edgelist(edgelist_gpu, source='src', destination='dst', store_transposed=True)

/usr/local/lib/python3.10/dist-packages/cugraph/structure/symmetrize.py:93: FutureWarning: Multi is deprecated and the removal of multi edges will no longer be supported from 'symmetrize'. Multi edges will be removed upon creation of graph instance.
  warnings.warn(


CPU times: user 240 ms, sys: 35.2 ms, total: 275 ms
Wall time: 266 ms


Pretvorba v graf je trajala 266ms kar je približno **296x hitreje** kot prej. Prepričajmo se, da je graf uspešno pripravljen za nadaljnjo obdelavo.

In [13]:
print('Graf cuGraph')
print(f'Vozlišča: {G_gpu.number_of_nodes()} \nPovezave: {G_gpu.number_of_edges()}')

Graf cuGraph
Vozlišča: 3774768 
Povezave: 16518948


### PageRank (cuGraph)

Izvedimo analizo grafov s cuGraph PageRank. Algoritem zaženimo s funkcijo [``pagerank()``](https://docs.rapids.ai/api/cugraph/stable/api_docs/api/cugraph/cugraph.pagerank/#cugraph.pagerank), pri tem pa merimo čas izvedbe.

In [14]:
%time pagerank_gpu = cg.pagerank(G_gpu)

CPU times: user 186 ms, sys: 48.2 ms, total: 234 ms
Wall time: 355 ms


/usr/local/lib/python3.10/dist-packages/cugraph/link_analysis/pagerank.py:227: UserWarning: Pagerank expects the 'store_transposed' flag to be set to 'True' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)


Izvedba algoritma PageRank je trajala 355ms kar je približno **287x hitreje** kot prej. Preverimo še 5 najbolje rangiranih vozlišč:

In [15]:
pagerank_gpu.sort_values(by='pagerank', ascending=False).head(5)

,vertex,pagerank
87,4237224,0.000087
186864,3813316,0.000075
7287,3932805,0.000043
12134,4395486,0.000039
2570,4298685,0.000036


### HITS (cuGraph)

Poskusimo še z algoritmom HITS, ki ga zaženemo s funkcijo [``hits()``](https://docs.rapids.ai/api/cugraph/stable/api_docs/api/cugraph/cugraph.hits/#cugraph.hits) in ponovno merimo čas izvedbe.

In [16]:
%time hits_gpu = cg.hits(G_gpu)

/usr/local/lib/python3.10/dist-packages/cugraph/link_analysis/hits.py:91: UserWarning: HITS expects the 'store_transposed' flag to be set to 'True' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)


CPU times: user 97.6 ms, sys: 8.98 ms, total: 107 ms
Wall time: 131 ms


Izvedba algoritma HITS je trajala 131ms kar je približno **755x hitreje** kot prej. Poglejmo še 5 vozlišč z najvišjo oceno avtoritete:

In [17]:
hits_gpu.sort_values(by='authorities', ascending=False).head(5)

,vertex,hubs,authorities
2876,5141797,1.996989e-08,0.001742
9340,4250096,2.580347e-09,0.001069
4485,4308400,5.927657e-11,0.001053
3870,4318791,6.050144e-05,0.001053
744,4058400,2.410486e-05,0.001052


Opazimo lahko **bistveno pohitritev** časa izvajanja tako nalaganja podatkovne zbirke in pretvorbe v graf kot tudi časa izvajanja algoritmov PageRank in HITS. Seveda pa knjižnica cuGraph ne podpira le teh dveh algoritmov, temveč tudi druge algoritme in operacije nad grafi. Seznam teh lahko podrobneje preučite v [dokumentaciji cuGraph](https://docs.rapids.ai/api/cugraph/stable/api_docs/).